In [1]:
import numpy as np
import tensorflow as tf
from tensorflow._api.v1.layers import *
import tensorlayer as tl
from Preprocessing import *
from metrics import *
from model import *

In [2]:
import dataset

28
13
28
(1, 240, 240, 153)
(1, 240, 240, 153)
(1, 240, 240, 153)
(1, 240, 240, 153)
{'DWI': {'mean': 21.078682, 'std': 44.95277}, 'Flair': {'mean': 24.302284, 'std': 50.49608}, 'T1': {'mean': 31.60154, 'std': 63.932735}, 'T2': {'mean': 44.153976, 'std': 92.98761}}
 HGG Validation
finished 12
finished 13
 HGG Train
finished 1
finished 2
finished 3
finished 4
finished 5
finished 6
finished 7
finished 8
finished 9
finished 10
finished 11
finished 12
(1836, 240, 240, 4)
(1836, 240, 240)
Preparing Testing Data
finished 14
finished 15
(306, 240, 240, 4)
(306, 240, 240)


In [3]:
task = "all"
modelName = "./checkpoint/unet_attention_{}.ckpt".format(all)

# FOLDERS Initialization
save_dir = "checkpoint"
tl.files.exists_or_mkdir(save_dir)
tl.files.exists_or_mkdir("test_samples/{}".format(task))

# Prepare Dataset
XTrain = dataset.X_train_input
yTrain = dataset.X_train_target[:, :, :, np.newaxis]
XTest = dataset.X_dev_input
yTest = dataset.X_dev_target[:, :, :, np.newaxis]
yTrain = (yTrain > 0).astype(int)
yTest = (yTest > 0).astype(int)

# Normalize data
XTrain, yTrain = NormalizeImageSet(XTrain, yTrain)
XTest, yTest = NormalizeImageSet(XTest, yTest)

# hyperparams 
batch_size = 10
lr = 0.0001
n_epoch = 120
evalEpoch = 100
dropout_value = 0.05

[TL] [!] checkpoint exists ...
[TL] [!] test_samples/all exists ...


In [4]:
tf.reset_default_graph()

In [5]:
tf.reset_default_graph()
with tf.device("/cpu:0"):
    sess = tf.Session(config = tf.ConfigProto(allow_soft_placement = True))
    with tf.device("/gpu:0"):
        
        XInput = tf.placeholder(dtype = tf.float32, shape = [batch_size, 240, 240, 4], name = "input_image")
        yInput = tf.placeholder(dtype = tf.float32, shape = [batch_size, 240, 240, 1], name = "ground_truth")
        dropout_ph = tf.placeholder_with_default(tf.cast(dropout_value, dtype = tf.float32), shape = [], name = "dropout_probability")
        
        yOutput = UNetWithAttention(XInput, reuse = False, n_out = 1, dropout_ph = dropout_ph, batchNorm = True)
        
        yOutputTest = UNetWithAttention(XInput, reuse = True, n_out = 1, dropout_ph = dropout_ph, batchNorm = True)
        
        # defining loss
        loss = 1 - tl.cost.dice_coe(yOutput, yInput, axis = [0, 1, 2, 3])
        testLoss = 1 - tl.cost.dice_coe(yOutputTest, yInput, axis = [0, 1, 2, 3])
    
    
    trainVar = tl.layers.get_variables_with_name('u_net', True, True)
    with tf.device('/gpu:0'):
        with tf.variable_scope('learning_rate'):
            lrVar = tf.Variable(lr, trainable = False)
        
        optimizer = tf.train.AdamOptimizer(lrVar).minimize(loss, var_list = trainVar)
    
    sess.run(tf.global_variables_initializer())
    # load existing model if exist
    saver = tf.train.Saver(var_list = trainVar)

 * Input: size of image: 240 240 4
 * Input: size of image: 240 240 4
[TL]   [*] geting variables with u_net
[TL]   got   0: u_net_attention/conv1_l1_1/kernel:0   (3, 3, 4, 64)
[TL]   got   1: u_net_attention/conv1_l1_1/bias:0   (64,)
[TL]   got   2: u_net_attention/batch_normalization/gamma:0   (64,)
[TL]   got   3: u_net_attention/batch_normalization/beta:0   (64,)
[TL]   got   4: u_net_attention/convl1_2/kernel:0   (3, 3, 4, 64)
[TL]   got   5: u_net_attention/convl1_2/bias:0   (64,)
[TL]   got   6: u_net_attention/batch_normalization_1/gamma:0   (64,)
[TL]   got   7: u_net_attention/batch_normalization_1/beta:0   (64,)
[TL]   got   8: u_net_attention/conv1_l2_1/kernel:0   (3, 3, 64, 128)
[TL]   got   9: u_net_attention/conv1_l2_1/bias:0   (128,)
[TL]   got  10: u_net_attention/batch_normalization_2/gamma:0   (128,)
[TL]   got  11: u_net_attention/batch_normalization_2/beta:0   (128,)
[TL]   got  12: u_net_attention/convl2_2/kernel:0   (3, 3, 64, 128)
[TL]   got  13: u_net_attention

In [9]:
m = {'Precision':[], 'Recall':[], 'Accuracy':[], 'F_score':[], 'Confusion':[[],[],[],[]]}
saver.restore(sess, tf.train.latest_checkpoint("./checkpoint"))
# EVALUATE
total_loss, n_batch = 0, 0
for batch in tl.iterate.minibatches(inputs=XTest, targets=yTest, batch_size=batch_size, shuffle=True):
    b_images, b_labels = batch
    _loss, out = sess.run([testLoss, yOutputTest],{XInput: b_images, yInput: b_labels})
    total_loss += _loss
    n_batch += 1
    for label,output in zip(b_labels, out):
        print(label.shape, output.shape, sep = "\n")
        metric(output, labels, m)

print(" **"+" "*17+"test -- loss: %f" %(total_loss/n_batch))
print(" task: {}".format(task))
m_avg = avg_metrics(m)
df = pd.DataFrame(data=[m_avg])

INFO:tensorflow:Restoring parameters from ./checkpoint\u_net_attention_all.ckpt
(240, 240, 1)
(240, 240, 1)


D:\Projects\Brain Tumor Research\Brain-Tumor-Attn-TF\metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  P_i = TP_i/(TP_i+FP_i) #Precision for ith test-reference sample
D:\Projects\Brain Tumor Research\Brain-Tumor-Attn-TF\metrics.py:21: RuntimeWarning: invalid value encountered in long_scalars
  R_i = TP_i/(TP_i+FN_i) #Recall for ith test-reference sample
D:\Projects\Brain Tumor Research\Brain-Tumor-Attn-TF\metrics.py:23: RuntimeWarning: invalid value encountered in long_scalars
  Accuracy_i = (TP_i+TN_i)/(TP_i+TN_i+FP_i+FN_i) #Accuracy for ith test-reference sample


(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 240, 1)
(240, 

KeyError: 0

In [8]:
m_avg

{'Accuracy_avg': nan,
 'Precision_avg': nan,
 'F_score_avg': nan,
 'Recall_avg': nan,
 'TP': 0.0,
 'FP': 0.0,
 'FN': 0.0,
 'TN': 0.0}

In [ ]:
sess.close()